In [21]:
import os
from enum import Enum
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mycolorpy import colorlist as mcp
from z_base import Experiment, Mols, T_ZERO
from scipy.stats import linregress

plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=15)
plt.rc('legend', fontsize=15)
plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams['figure.subplot.left'] = 0.12
plt.rcParams['figure.subplot.right'] = 0.99
plt.rcParams['figure.subplot.top'] = 0.99
plt.rcParams['figure.subplot.bottom'] = 0.1

In [2]:
%matplotlib qt

In [57]:
DF= pd.read_csv(r'Datas\MD_viscosity.csv',index_col=0)
DF.head(3)

,compound,T,Nu,rho,w_mass,w_vol,w,V,ln_Nu,T_1,D,ln_D,D_comp,ln_D_comp,D_but,ln_D_but
0,butanol,286.15,3.72,0.82668,100.0,100.0,100.0,4.499927,0.570543,0.003495,76.922043,1.886051,1.892807e-10,-9.722894,1.892807e-10,-9.722894
1,butanol,286.35,3.70,0.82668,100.0,100.0,100.0,4.475734,0.568202,0.003492,77.391892,1.888695,1.904369e-10,-9.720249,1.904369e-10,-9.720249
2,butanol,286.45,3.70,0.82668,100.0,100.0,100.0,4.475734,0.568202,0.003491,77.418919,1.888847,1.905034e-10,-9.720097,1.905034e-10,-9.720097


In [68]:
class W(Enum):
    w = "w mol  %"
    w_mass = "w mass  %"
    w_vol = "w vol  %"
    T = 'Temperature  C'
    Nu = 'Viscosity  cP'
    ln_Nu = 'ln Viscosity'
    V = 'Kinematic viscosity cCt'
    T_1 = '1000/Temperature  K$^{-1}$'
    #
    D_comp = 'Diffusion*1e6   nm2/s '
    D_but = 'Diffusion*1e6  nm2/s'
    ln_D_comp = 'log Diffusion'
    ln_D_but = 'log Diffusion '

In [71]:
# Comparation pure
x_i = W.T_1
y_i = W.ln_D_comp
g_i = W.w_mass

compound_filter = (DF["w"] == 100) | (DF["w"] == 0)
df =DF[compound_filter]
data = []

fig, ax = plt.subplots(num="DT")
for (compound, *_), group in df.groupby(by=['compound']):
    y = group[y_i.name]
    T = group[x_i.name]
    ax.scatter(T*1000, y, s=5)

    R = linregress(T, y)
    line_x = np.array([0.00315, 0.0035])
    ax.plot(
        line_x*1000,
        R.slope * line_x + R.intercept,
        linewidth=1,
        label=f"{compound:5}",
    )
ax.set_xlabel(x_i.value)
ax.set_ylabel(y_i.value)

plt.legend()
plt.show()

In [91]:
# Grouped compound Y(x)
compound = Mols.ocm
x_i = W.T_1
y_i = W.ln_D_comp
y2_i = W.ln_D_but

filt = (DF["compound"] == compound.name) & np.isclose(DF['w_mass'],2.39,atol=0.1)
group = DF[filt]

fig, ax = plt.subplots(num="DT")



y = group[y_i.name]
y2=group[y2_i.name]

T = group[x_i.name]

ax.scatter(T*1000, y, s=2,c='green',)

R = linregress(T, y)
line_x = np.array([0.00315, 0.0035])
ax.plot(
    line_x*1000,
    R.slope * line_x + R.intercept,
    linewidth=1,
    c='green',
    label=f"OCM-2",
)

ax.scatter(T*1000, y2, s=2,c='blue',)

R = linregress(T, y2)
line_x = np.array([0.00315, 0.0035])
ax.plot(
    line_x*1000,
    R.slope * line_x + R.intercept,
    linewidth=1,
    c='blue',
    label=f"Butanol-1",
)

ax.set_xlabel(x_i.value)
ax.set_ylabel(y_i.value)

plt.legend()
plt.show()

In [83]:
# Window Y(x)
compound = Mols.ocm
x_i = W.w
y_i = W.ln_D_comp

compound_filter = (DF["compound"] == compound.name)
# | (DF["compound"] == Mols.butanol.name)
grouped = DF[compound_filter]
T=25+T_ZERO



cond = (T - 2.5 <= grouped["T"]) & (grouped["T"] <= T + 2.5)
group2= grouped[cond]
group = group2[[x_i.name, y_i.name]].groupby(by=x_i.name).mean().reset_index()

x = group[x_i.name]
y = group[y_i.name]

fig, ax = plt.subplots(num="DX")
ax.scatter(x, y,  label=f"OCM-2")
line_x=np.array([0,60])
r=linregress(x,y)
ax.plot(line_x,r.slope*line_x+r.intercept,)

# ax.set_title(f"{compound.value.upper()}")
ax.set_xlabel(f"{x_i.value} [%]")
ax.set_ylabel(y_i.value)
plt.legend()
plt.show()

In [ ]:
# E
x_i = W.x
y_i = W.ln_D

fig, ax = plt.subplots(num="E")
for (compound, *_), group in DF.groupby(by=["compound"]):
    y = []
    x = []
    for (x_val, *_), sub_group in group.groupby(by=[x_i.name]):
        r = linregress(sub_group[['T_1', y_i.name]])
        y.append(r.slope* 8.314 / 1000)
        x.append(x_val)
    ax.plot([x[0],100], [y[0],-19.73])
    ax.scatter(x, y, label=f"{compound}")

# ax.set_title(f"E {y_i.value}")
ax.set_xlabel(f"{x_i.value}")
ax.set_ylabel("E  kJ/mol")
plt.legend()
plt.show()